# Data prepare and FC_DNN

In [1]:
# Import data_prepare modul that contains functions that we created.
from data_prepare import *
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from scripts.reconstruction_minimal import createAudio
import scipy.io.wavfile as wavfile

# Prepare data for learning

We prepare the data for learning neural networks. We use a function that execute the results of the last milestone get a train-ready dataset.

In [2]:
def generate_datasets(pt, feat_path=r'./features', cnn_prepare=False):
    
    r"""
    Generate train and test datasets from the one sample of the raw data.
    param pt: name of the sample
    param feat_path: path of the raw data file
  """
    
    spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))
    data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))
    
    # Reduce Dimensions
    pca = PCA()
    pca.fit(data)
    data = np.dot(data, pca.components_[:50,:].T)
    
    if cnn_prepare:
        # Creating data for CNN: stacking window_size input data and connecting it to the output
        # at the end of the window
        window_size = 4
        input = np.zeros((data.shape[0] - window_size, window_size, data.shape[1]))
        output = np.zeros((spectrogram.shape[0] - window_size, spectrogram.shape[1]))
        for i in range(input.shape[0] - window_size):
          output[i, :] = spectrogram[i + window_size, :]
          input[i, :, :] = data[i:i+window_size, :]
    else:
        output = spectrogram
        input = data

    # Generate train, validation and test datasets
    X_train, X_test, Y_train, Y_test = train_test_split(input, output, test_size=0.1, shuffle=False)

    # Shuffle train data
    randomize = np.arange(len(X_train))
    np.random.shuffle(randomize)
    X_train = X_train[randomize]
    Y_train = Y_train[randomize]
    
    # Standardize input
    mu_input = np.mean(X_train, axis=0)
    std_input = np.std(X_train, axis=0)
    X_train = (X_train-mu_input)/std_input
    X_test = (X_test-mu_input)/std_input

    # Standardize output
    mu_output = np.mean(Y_train, axis=0)
    std_output = np.std(Y_train, axis=0)
    Y_train = (Y_train-mu_output)/std_output
    Y_test = (Y_test-mu_output)/std_output


    return (X_train, Y_train, X_test, Y_test, mu_input, std_input, mu_output, std_output)

In [3]:
def generate_merged_dataset(cnn_prepare=False):
    norm_params = {}
    X_test = []
    Y_test = []
    pts = ['sub-%02d'%i for i in range(1,11)]
    (X_train, Y_train, x_test, y_test, mu_input, std_input, mu_output, std_output) = generate_datasets(pts[0], cnn_prepare=cnn_prepare)
    norm_params[str(0)]={'mu_input':mu_input, 'std_input':std_input, 'mu_output':mu_output, 'std_output':std_output}
    X_test.append(x_test)
    Y_test.append(y_test)
    for i, pt in enumerate(pts[1:]):
        (x_train, y_train, x_test, y_test, mu_input, std_input, mu_output, std_output) = generate_datasets(pt, cnn_prepare=cnn_prepare)
        norm_params[str(i+1)]={'mu_input':mu_input, 'std_input':std_input, 'mu_output':mu_output, 'std_output':std_output}
        X_train = np.concatenate((X_train, x_train))
        Y_train = np.concatenate((Y_train, y_train))
        X_test.append(x_test)
        Y_test.append(y_test)
    return (X_train, Y_train, X_test, Y_test, norm_params)

# Create FC-DNN network

In [4]:
def create_fndnn(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(units=150, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=80, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=60, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=30, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=output_dim, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    return model

In [5]:
def make_1d_convnet(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=[40, 40]):
    #probably more filter layers should be added

    model = Sequential()
    model.add(Conv1D(filters=nb_filter[0], kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)))
    model.add(Conv1D(filters=nb_filter[1], kernel_size=filter_length, activation='relu'))
    model.add(Flatten())
    model.add(Dense(nb_outputs, activation='linear'))

    optimizer=Adam(learning_rate=0.001)
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [6]:
def test_model(model, X_test, Y_test, norm_params):
    preds = []
    spectograms = []
    losses = []
    for i in range(10):
        y_pred = model.predict(X_test[i])
        losses.append(mean_absolute_error(Y_test[i], y_pred))
        preds.append(y_pred * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output'])
        spectograms.append(Y_test[i] * norm_params[str(i)]['std_output'] + norm_params[str(i)]['mu_output'])
    mae = np.sum(losses)
    print(f'Mean absolute error of test set:{mae}')
    return preds, spectograms

In [7]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_merged_dataset(cnn_prepare=True)

#Parameters for the CNN network
# one data point covers 2 * 200 ms of eeg signal
filter_length = 2 #covers 4*8*400 sec of eeg
window_size = 2 * 2 #we have two 1d max pool layers
nb_filter = [40, 40]
epochs = 1000
batch_size = 512

early_stopping=EarlyStopping(patience=50, verbose=1)
checkpointer=ModelCheckpoint(filepath='cnn.hdf5', save_best_only=True, verbose=1)
model_cnn = make_1d_convnet(window_size=window_size, filter_length=filter_length, nb_filter=nb_filter, nb_input_series=50, nb_outputs=23)
history = model_cnn.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping])

preds_cnn, spectograms = test_model(model_cnn, X_test, Y_test, norm_params)

Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.90327, saving model to cnn.hdf5
472/472 - 5s - loss: 0.9280 - mae: 0.7285 - val_loss: 0.9033 - val_mae: 0.6071 - 5s/epoch - 10ms/step
Epoch 2/1000

Epoch 2: val_loss did not improve from 0.90327
472/472 - 3s - loss: 0.8281 - mae: 0.6709 - val_loss: 0.9385 - val_mae: 0.6163 - 3s/epoch - 7ms/step
Epoch 3/1000

Epoch 3: val_loss did not improve from 0.90327
472/472 - 3s - loss: 0.7840 - mae: 0.6440 - val_loss: 0.9809 - val_mae: 0.6334 - 3s/epoch - 7ms/step
Epoch 4/1000

Epoch 4: val_loss did not improve from 0.90327
472/472 - 3s - loss: 0.7586 - mae: 0.6292 - val_loss: 0.9670 - val_mae: 0.6168 - 3s/epoch - 7ms/step
Epoch 5/1000

Epoch 5: val_loss did not improve from 0.90327
472/472 - 3s - loss: 0.7405 - mae: 0.6189 - val_loss: 0.9523 - val_mae: 0.6158 - 3s/epoch - 7ms/step
Epoch 6/1000

Epoch 6: val_loss did not improve from 0.90327
472/472 - 3s - loss: 0.7256 - mae: 0.6099 - val_loss: 0.9589 - val_mae: 0.6184 - 3s/epoch - 7ms/step
E

KeyboardInterrupt: 

In [ ]:
(X_train, Y_train, X_test, Y_test, norm_params) = generate_merged_dataset()

#Parameters for the FC-DNN network
epochs = 1000
batch_size = 512

early_stopping=EarlyStopping(patience=50, verbose=1)
checkpointer=ModelCheckpoint(filepath='fcdnn.hdf5', save_best_only=True, verbose=1)
model_fcdnn = create_fndnn(input_dim=X_train.shape[1], output_dim=Y_train.shape[1])
history = model_fcdnn.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[checkpointer, early_stopping])

preds_fcdnn, spectograms = test_model(model_fcdnn, X_test, Y_test, norm_params)

Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.99922, saving model to fcdnn.hdf5
472/472 - 8s - loss: 1.0102 - mae: 0.7779 - val_loss: 0.9992 - val_mae: 0.6534 - 8s/epoch - 17ms/step
Epoch 2/1000

Epoch 2: val_loss improved from 0.99922 to 0.99822, saving model to fcdnn.hdf5
472/472 - 7s - loss: 0.9993 - mae: 0.7748 - val_loss: 0.9982 - val_mae: 0.6473 - 7s/epoch - 14ms/step
Epoch 3/1000

Epoch 3: val_loss improved from 0.99822 to 0.98615, saving model to fcdnn.hdf5
472/472 - 6s - loss: 0.9880 - mae: 0.7685 - val_loss: 0.9862 - val_mae: 0.6251 - 6s/epoch - 13ms/step
Epoch 4/1000

Epoch 4: val_loss improved from 0.98615 to 0.97740, saving model to fcdnn.hdf5
472/472 - 7s - loss: 0.9747 - mae: 0.7585 - val_loss: 0.9774 - val_mae: 0.6152 - 7s/epoch - 14ms/step
Epoch 5/1000

Epoch 5: val_loss improved from 0.97740 to 0.97543, saving model to fcdnn.hdf5
472/472 - 7s - loss: 0.9639 - mae: 0.7518 - val_loss: 0.9754 - val_mae: 0.6073 - 7s/epoch - 14ms/step
Epoch 6/1000

Epoch 6: val_lo

In [ ]:
# Viz spectrograms
def visualize_results(preds, spectograms):

    cm='viridis'
    fig, ax = plt.subplots(2, sharex=True)

    for i in range(10):
        #Plot spectrograms
        ax[0].imshow(np.flipud(spectograms[i].T), cmap=cm, interpolation=None,aspect='auto')
        ax[0].set_ylabel('Log Mel-Spec Bin')
        ax[1].imshow(np.flipud(preds[i].T), cmap=cm, interpolation=None,aspect='auto')
        ax[1].set_ylabel('Log Mel-Spec Bin')

In [ ]:
# Synthesize audio files
def synthesize_audio(preds, spectograms, result_path):
    winLength = 0.05
    frameshift = 0.01
    audiosr = 16000
    if not os.path.isdir(result_path):
        os.makedirs(result_path)
    for i in range(10):
        #Synthesize waveform from spectrogram using Griffin-Lim
        reconstructedWav = createAudio(preds[i],audiosr=audiosr,winLength=winLength,frameshift=frameshift)
        wavfile.write(os.path.join(result_path,f'{i+1}_predicted.wav'),int(audiosr),reconstructedWav)

        #For comparison synthesize the original spectrogram with Griffin-Lim
        origWav = createAudio(spectograms[i],audiosr=audiosr,winLength=winLength,frameshift=frameshift)
        wavfile.write(os.path.join(result_path,f'{i+1}_orig_synthesized.wav'),int(audiosr),origWav)

In [ ]:
visualize_results(preds=preds_fcdnn, spectograms=spectograms)
visualize_results(preds=preds_cnn, spectograms=spectograms)

In [ ]:
synthesize_audio(preds=preds_fcdnn, spectograms=spectograms, result_path = 'dnn_results')
synthesize_audio(preds=preds_cnn, spectograms=spectograms, result_path = 'cnn_results')